## Deep Learning - Artificial Neural Networks

In [1]:
#importing libraries

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
#importing dataset
dataset=pd.read_csv('./../0. DataSets/Churn_Modelling.csv')
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
#Faz com que os displays (tanto do pandas como dos arrays tenham notacao decimal)
dataset.round()
pd.options.display.float_format = '{:,.0f}'.format
float_formatter = lambda x: "%.2f" % x
np.set_printoptions(formatter={'float_kind':float_formatter})

## Business Problem explanation:

In [4]:
#we have a list of customers from a bank with loads of different independent variables.
#based on these variables we want to predict the last column, that is, if the customer left/exited the bank or not

In [5]:
#Create X matrix with all the independent variables
#Create y vector with all the dependent variables
X=dataset.iloc[:,3:13].values #ignored the first columns, ie. customer id, surname
y=dataset.iloc[:,13].values

In [6]:
X

array([[619, 'France', 'Female', ..., 1, 1, 101348.88],
       [608, 'Spain', 'Female', ..., 0, 1, 112542.58],
       [502, 'France', 'Female', ..., 1, 0, 113931.57],
       ...,
       [709, 'France', 'Female', ..., 0, 1, 42085.58],
       [772, 'Germany', 'Male', ..., 1, 0, 92888.52],
       [792, 'France', 'Female', ..., 1, 0, 38190.78]], dtype=object)

In [7]:
#Encoding categorical data
#We only have 2 categorical variables: Geography and Gender. We have to create 2 different encoders

from sklearn.preprocessing import LabelEncoder, OneHotEncoder

#encode Geography, that means turn a word and turn into a number. On this case France will be turned into 0 and Spain will be turned into 2
labelencoder_X_1=LabelEncoder()
X[:,1]=labelencoder_X_1.fit_transform(X[:,1])

#encode Gender
labelencoder_X_2=LabelEncoder()
X[:,2]=labelencoder_X_2.fit_transform(X[:,2])

#Vamos onehotencode coluna 1. Que e geography.
#One hot encoded columns passam para o inicio da matrix (deixa de ser o credit score que agora esta na posicao 3)
onehotencoder=OneHotEncoder(categorical_features=[1])
X=onehotencoder.fit_transform(X).toarray()


In [8]:
#como fizemos one hot encode e ficamos com tres colunas (correnspondendo a spain,france e outro pais)
#podemos eliminar uma coluna para nao cair na dummy variable trap. ficando so com 2 dummy variables
X=X[:,1:]
X

array([[0.00, 0.00, 619.00, ..., 1.00, 1.00, 101348.88],
       [0.00, 1.00, 608.00, ..., 0.00, 1.00, 112542.58],
       [0.00, 0.00, 502.00, ..., 1.00, 0.00, 113931.57],
       ...,
       [0.00, 0.00, 709.00, ..., 0.00, 1.00, 42085.58],
       [1.00, 0.00, 772.00, ..., 1.00, 0.00, 92888.52],
       [0.00, 0.00, 792.00, ..., 1.00, 0.00, 38190.78]])

In [9]:
#Now, we have to split the dataset into the training set/test set
# from sklearn.cross_validation import train_test_split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=0)


In [10]:
#Feature scaling
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)

In [11]:
#Make the Artificial Neural Network

#Importing Keras libraries and required packages
import keras
from keras.models import Sequential
from keras.layers import Dense


Using TensorFlow backend.


In [17]:
#Initialising the ANN
classifier=Sequential()

#Adding the input layer and the first hidden layer
classifier.add(Dense(output_dim=6, init='uniform', activation='relu',input_dim=11))
#So precisamos de definir o input layer no primeiro layer, dps disso a ANN descobre automaticamente


#uma tip que deram para o output_dim e a media da soma dos inputs parameters (11) com os output parameters (1).
#neste caso (11+1) / 2 = 6

#the inits and activations and optimizers and losses are different if instead of a binary classification we had more categories
#Adding the second hidden layer
classifier.add(Dense(output_dim=6, init='uniform', activation='relu'))

#Adding the output layer
classifier.add(Dense(output_dim=1, init='uniform', activation='sigmoid'))

#Compiling the ANN 
classifier.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])

/home/xbuns/anaconda3/envs/my_env_1/lib/python3.6/site-packages/ipykernel/__main__.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=11, units=6, kernel_initializer="uniform")`
/home/xbuns/anaconda3/envs/my_env_1/lib/python3.6/site-packages/ipykernel/__main__.py:14: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=6, kernel_initializer="uniform")`
/home/xbuns/anaconda3/envs/my_env_1/lib/python3.6/site-packages/ipykernel/__main__.py:17: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="sigmoid", units=1, kernel_initializer="uniform")`


In [18]:
#Fitting the ANN to the training set
classifier.fit(X_train,y_train,batch_size=10, nb_epoch=100)

/home/xbuns/anaconda3/envs/my_env_1/lib/python3.6/site-packages/ipykernel/__main__.py:2: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  from ipykernel import kernelapp as app


Epoch 1/100
8000/8000 [==============================] - 2s 269us/step - loss: 0.4908 - acc: 0.8006
Epoch 2/100
8000/8000 [==============================] - 1s 177us/step - loss: 0.4044 - acc: 0.8269
Epoch 3/100
8000/8000 [==============================] - 2s 188us/step - loss: 0.3915 - acc: 0.8306
Epoch 4/100
8000/8000 [==============================] - 2s 194us/step - loss: 0.3825 - acc: 0.8312
Epoch 5/100
8000/8000 [==============================] - 2s 192us/step - loss: 0.3762 - acc: 0.8425
Epoch 6/100
8000/8000 [==============================] - 2s 236us/step - loss: 0.3705 - acc: 0.8470
Epoch 7/100
8000/8000 [==============================] - 2s 224us/step - loss: 0.3663 - acc: 0.8486
Epoch 8/100
8000/8000 [==============================] - 2s 199us/step - loss: 0.3626 - acc: 0.8500
Epoch 9/100
8000/8000 [==============================] - 1s 181us/step - loss: 0.3603 - acc: 0.8502
Epoch 10/100
8000/8000 [==============================] - 2s 207us/step - loss: 0.3580 - acc: 0.8542

8000/8000 [==============================] - 2s 211us/step - loss: 0.3386 - acc: 0.8619
Epoch 83/100
8000/8000 [==============================] - 2s 228us/step - loss: 0.3377 - acc: 0.8622
Epoch 84/100
8000/8000 [==============================] - 2s 206us/step - loss: 0.3392 - acc: 0.8626
Epoch 85/100
8000/8000 [==============================] - 2s 214us/step - loss: 0.3397 - acc: 0.8612
Epoch 86/100
8000/8000 [==============================] - 2s 216us/step - loss: 0.3379 - acc: 0.8627
Epoch 87/100
8000/8000 [==============================] - 2s 211us/step - loss: 0.3388 - acc: 0.8591
Epoch 88/100
8000/8000 [==============================] - 2s 210us/step - loss: 0.3377 - acc: 0.8597
Epoch 89/100
8000/8000 [==============================] - 2s 202us/step - loss: 0.3384 - acc: 0.8624
Epoch 90/100
8000/8000 [==============================] - 2s 197us/step - loss: 0.3377 - acc: 0.8620
Epoch 91/100
8000/8000 [==============================] - 2s 212us/step - loss: 0.3377 - acc: 0.8600
Epo

In [19]:
#agora que treinei o modelo vou usar a matrix X_test para o testar.
y_pred=classifier.predict(X_test)
y_pred

In [22]:
#para fazer a confusion matrix escolho um treshold onde digo que acima desse valor retorna true (neste caso o cliente deixou o banco)
y_pred=(y_pred>0.5)
y_pred

array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

In [26]:
from sklearn.metrics import confusion_matrix

#vou comparar os resultados do meu modelo com os resultados reais fazendo uma confusion matrix
cm=confusion_matrix(y_test,y_pred)

In [24]:
cm

array([[1482,  113],
       [ 173,  232]])

retornou isto:

array([[1482,  113],
       [ 173,  232]])

ou seja o meu modelo preveu que 1482 nao se iam embora e nao foram
232 pessoas iam embora e foram

falsos:
173 nao iam embora e foram 
113 iam embora e nao foram

In [25]:
#calcular a accuracy do modelo

(1482+232)/(1482+113+173+232)

0.857